# Agent Architectures in Practice — Multi‑Framework Lab (Fixed)
This version is organized into clear sections with code that **runs** (with graceful fallbacks).  
If you haven't installed the optional libraries yet, the cells will skip those sections and explain what to install.

## 0) Setup

In [ ]:
# Install (uncomment if needed). Run this cell, then restart the kernel.
# Note: package names are intentionally explicit to avoid version confusion.
# !pip install -U langchain langchain-openai langchain-community crewai pyautogen chromadb

import os

# Read API keys if available (optional for sections that use OpenAI)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if not OPENAI_API_KEY:
    print("Tip: Set OPENAI_API_KEY in your environment if you want to use OpenAI-backed sections.")

## 1) CrewAI — Role-based Agents

In [ ]:
try:
    from crewai import Agent, Task, Crew, Process
    # Optional LLM via LangChain (CrewAI can also use other providers)
    from langchain_openai import ChatOpenAI

    llm = None
    if os.getenv("OPENAI_API_KEY"):
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    else:
        print("OPENAI_API_KEY not set — CrewAI will run with no LLM (agents will be configured but kickoff may be limited).")

    researcher = Agent(
        role="Researcher",
        goal="Analyze LLMs and extract pros/cons for GPT-4, Claude, and Mistral.",
        backstory="You specialize in balanced evaluations of foundation models.",
        verbose=True,
        llm=llm
    )

    writer = Agent(
        role="Writer",
        goal="Summarize findings into a concise comparison table and bullet list.",
        backstory="You convert technical findings into clear outputs for executives.",
        verbose=True,
        llm=llm
    )

    task_research = Task(
        description="Research the strengths, weaknesses, and ideal use-cases of GPT-4, Claude, and Mistral. Return ~6 bullets per model.",
        agent=researcher,
        expected_output="A structured bullet list per model with at least one risk/limitation called out."
    )

    task_write = Task(
        description="Turn the research into a one-paragraph summary + a compact table with at least 4 comparison dimensions.",
        agent=writer,
        expected_output="One paragraph + a Markdown table. Keep it neutral and cite sources if any were used."
    )

    crew = Crew(
        agents=[researcher, writer],
        tasks=[task_research, task_write],
        process=Process.sequential
    )

    print("CrewAI: constructed crew with 2 agents and 2 tasks.")
    # Running kickoff requires an LLM; if no key is set, we skip actual execution.
    if llm:
        result = crew.kickoff()
        print("\n=== CrewAI result (truncated) ===\n")
        print(str(result)[:1200])
    else:
        print("Skipped kickoff because OPENAI_API_KEY was not provided.")
except ModuleNotFoundError as e:
    print("CrewAI section skipped. Missing package:", e.name)
    print("Install with: pip install crewai langchain-openai")

## 2) LangChain — Tool‑Using Agent (Zero‑Shot ReAct)

In [ ]:
try:
    from langchain.agents import initialize_agent, AgentType, Tool
    from langchain_openai import ChatOpenAI

    # A simple stub tool; replace with a real search tool if desired.
    def web_search(query: str) -> str:
        return f"[stub] Search result for: {query}"

    tools = [
        Tool(
            name="web_search",
            func=web_search,
            description="Use this to look up facts before answering."
        )
    ]

    llm = None
    if os.getenv("OPENAI_API_KEY"):
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    else:
        print("OPENAI_API_KEY not set — using a tool-only skeleton; agent call will be skipped.")

    if llm:
        agent = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
        )
        response = agent.run("Compare GPT-4, Claude, and Mistral in 5 bullets each.")
        print(response)
    else:
        # Even without LLM, we can show the tool works
        print(web_search("LLM comparison dimensions"))
except ModuleNotFoundError as e:
    print("LangChain section skipped. Missing package:", e.name)
    print("Install with: pip install langchain langchain-openai langchain-community")

## 3) AutoGen — Two‑Agent Chat

In [ ]:
try:
    from autogen import AssistantAgent, UserProxyAgent

    if not os.getenv("OPENAI_API_KEY"):
        print("OPENAI_API_KEY not set — AutoGen demo will define agents but skip the conversation.")
        config_list = []
    else:
        # Minimal config_list for OpenAI-compatible providers
        config_list = [{
            "model": "gpt-4o-mini",
            "api_key": os.getenv("OPENAI_API_KEY")
        }]

    assistant = AssistantAgent(
        name="assistant",
        llm_config={
            "config_list": config_list,
            "temperature": 0
        }
    )
    user_proxy = UserProxyAgent(
        name="user",
        human_input_mode="NEVER",   # non-interactive for notebooks
        code_execution_config={"use_docker": False}
    )

    if config_list:
        user_proxy.initiate_chat(
            assistant,
            message="Give a concise comparison of GPT-4, Claude, and Mistral (pros/cons & best-fit scenarios)."
        )
    else:
        print("Skipped AutoGen chat because OPENAI_API_KEY was not provided.")
except ModuleNotFoundError as e:
    print("AutoGen section skipped. Missing package:", e.name)
    print("Install with: pip install pyautogen")

## 4) (Optional) LangGraph Sketch — Pseudo‑code

In [ ]:
# Pseudo-code only (no execution):
# from langgraph.graph import StateGraph
# graph = StateGraph()
# graph.add_node("start", StartNode())
# graph.add_node("search", SearchNode())
# graph.add_node("reflect", ReflectNode())
# graph.set_entry_point("start")
# graph.add_edge("start", "search")
# graph.add_edge("search", "reflect")
# graph.add_edge("reflect", "search")  # retry loop